# Understanding Retrivals

- Suff Method
- MapReduce Method 

In [98]:
!pip install groq sentence-transformers langchain_community faiss-cpu langchain-text-splitters unstructured


In [99]:
import os
from groq import Groq
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS


In [100]:
#load openAI api key
os.environ["GROQ_API_KEY"] = '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'
client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [101]:
loaders = UnstructuredURLLoader(
    urls = [
       'https://economictimes.indiatimes.com/markets/stocks?from=mdr'
        'https://www.sciencedirect.com/topics/social-sciences/stock-price'
        ] 
)
data = loaders.load() 
len(data)

1

In [102]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " "]
)

chunks = splitter.split_documents(data)
len(chunks)


60

In [103]:
model = SentenceTransformer("all-MiniLM-L6-v2")

texts = [c.page_content for c in chunks]
vectors = model.encode(texts, show_progress_bar=True)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [104]:
!pip install faiss-cpu


In [105]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


In [106]:
model = SentenceTransformer("all-MiniLM-L6-v2")
texts = [c.page_content for c in chunks]
embeddings = model.encode(texts)
embeddings = np.array(embeddings).astype("float32")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [107]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("FAISS index created!")


FAISS index created!


In [108]:
def ask_groq(question, context):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"Context:\n{context}\n\nQuestion: {question}"
            }
        ]
    )
    return response.choices[0].message.content


In [111]:
query = "What about the news of latest stock ?"

# Query embedding
q_emb = model.encode([query]).astype("float32")

# Search FAISS
distances, ids = index.search(q_emb, k=3)

# Build context from best matches
context = "\n\n".join([texts[i] for i in ids[0]])

# Ask Groq using your function
answer = ask_groq(query, context)
print(answer)


According to the given context, here are some of the latest stock-related news updates:

1. Nifty saw profit-taking after a gap-up opening but held above key Fibonacci and moving average supports. Analysts flagged a shift to buy-on-dips, cooling volatility and improving momentum, even as global markets weakened and select heavyweights influenced overall market sentiment.

2. Sensex settled 2,073 points higher due to trade deal euphoria, and Nifty rose above 25,700.

3. Adani Ports rose 9% and Bajaj Finance rose 7%.

However, there are specific stock-related news updates as follows:

- Interglobe Aviation (IndiGo) and Adani Energy Solutions both announced their Q3 results for FY26, which might have an impact on the stock market.

- Reliance Industries, Reliance Jio, Wipro, Meesho, HAL, JP Power, Waaree Energies, and Infosys are among the other stocks mentioned in the context with updates on share prices, IPO allotment, and share buybacks.

It is recommended to check the latest market ne